# The Optimal Locations for Indian Restaurent

## Applied Data Science Capstone by IBM/Coursera



## Table of Contents

* [Introducation to Bussiness problem](#introducation)
* [Data]()
* [Methodology]()
* [Result]()
* [Conclusion](#clonclustion)

## Introducation to Bussiness problem<a name="introduction"></a>

As an Indian and food lover, I was always curious to, what if someone like me want to open Indian restaurant. Great!!! 
I choose London for my analysis. London is diverse city. London is home of 500k people roughly 10% of total population, originated from Indian, Pakistan and Bangladesh. India Pakistan and Bangladesh were a single country in colonial era. And it had been ruled by Britain, so lot of people for Indian subcontinent are moved to Britain and especially London, Additional to that, lot of travellers visit to London. Thant’s why I am so excited about London

Since we have decided to open an Indian restaurant in London, we need to get optimal location. For that we are considering some criteria. 
•	We will consider the area which has more Indian population
•	Restaurant should be not crowed with another Indian restaurants
•	It should be as near to city centre as possible
There are 33 boroughs in London, and we will study each of them


In [2]:
!pip install geopy 



#### 1. Importing Libraries

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
sns.set(style='darkgrid')

import pandas.io.json
import requests
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim
from  geopy import distance

import geocoder
import folium

# Hide all warnings
import warnings
warnings.filterwarnings('ignore')

print('Libraries Imported')

ModuleNotFoundError: No module named 'geocoder'


## 2. Data
   We will work in Borough of London, UK, we will scrap the data from [List of London Borough](https://en.wikipedia.org/wiki/List_of_London_boroughs). Page contens two table 'List of boroughs and local authorities' and
   'City of London'

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs',header=0)[0]

In [ ]:
df

In [ ]:
df2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs',header=0)[1]

In [ ]:
df = df.append(df2, ignore_index=True)

In [ ]:
df

In [ ]:
df['Population (2013 est)[1]'][32] = df['Population(2011 est)'][32]

In [ ]:
df

####  Plot the Borough on Map

In [ ]:
# define a function to get coordinates
def get_latlng1(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London,UK '.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

Let's take a Indian population, I will take down the population manually from wikipedia page of every borough, Because it was little bit harder for me to scrab.
I will consider Indian, Pakistani, and Bangladeshi as Indian pople as the food is simillar.

In [ ]:
indian_population = [23144, 35479,8554,74147,8494,20074,38095,64737,19841, 12543,15684,6119, 12282,72226, 8046,48634,64026,9147,4324,10226,
10276,7584,17659, 110053, 92722, 7722, 11354, 10232, 90606, 40113, 19853, 15840, 464]

In [ ]:
loaded = False
try:
    London_df = pd.read_csv('London_df.csv')
    loaded = True
except:
    pass
    

In [ ]:

if not loaded:  
    #first we need to get the co-ordinated of London,UK
    g = geocoder.arcgis( 'London, UK ')
    coordinated_centerlondon = g.latlng

    coords_london = [ get_latlng1(neighborhood) for neighborhood in df["Borough"].tolist() ]
    df['LatLng'] = coords_london
    df['Lat'] = df['LatLng'].apply(lambda x:x[0])
    df['Lng'] = df['LatLng'].apply(lambda x:x[1])
    London_df = df[['Borough','Lat','Lng','Area (sq mi)','Population (2013 est)[1]']]
    London_df.columns = ['Borough','Lat','Lng','Area','Population']
    London_df.Borough = London_df['Borough'].apply(lambda x:x.split('[')[0])
    
    #adding Indian population to Data frames
    London_df['Indian_population'] = indian_population    
    
    London_df['Dist_from_center'] = np.NaN
    for index in range(London_df.shape[0]):
        coords_1 = coordinated_centerlondon
        coords_2 = (London_df['Lat'].iloc[index],London_df['Lng'].iloc[index])
        
        # get the of borough from center of the london city
        London_df['Dist_from_center'].iloc[index] = distance.distance(coords_1, coords_2).km
    London_df.to_csv(r'C:\Users\Shrirang\Desktop\Python Practice\Online Courses\IMB Data Science Cousre\Capstone_Project\Coursera_Capstone\Capstone_project\London_df.csv',index= False)

In [ ]:
London_df.head()

In [ ]:
#lets get the center location of London

In [ ]:
g = geocoder.arcgis(' London,UK ')
coordinated_centerlondon = g.latlng

In [ ]:
coordinated_centerlondon

In [ ]:
#Create map of London and superinpose the Neighborhood on it
map_la = folium.Map(location = coordinated_centerlondon, zoom_start = 10)
folium.Marker(coordinated_centerlondon, popup='London_center').add_to(map_la)
for lat, lng in zip(London_df['Lat'], London_df['Lng']):
    #folium.CircleMarker([lat, lng], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_la)
    folium.Circle([lat,lng], radius=30,color ="blue").add_to(map_la)
map_la

### 2.1. Data from Foursquere API

Foursquare API is excelent 

In [ ]:
#foruesqure credentials
client_id = 'XXX'
client_secret = 'XXX'
version = '20200724'

In [ ]:
# this id's are aquired from Foursqure
food_category = '4d4b7105d754a06374d81259'
indian_Restaurant =     ['4bf58dd8d48988d10f941735','54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dff3',
                         '54135bf5e4b08f3d2429dff5','54135bf5e4b08f3d2429dfe2','54135bf5e4b08f3d2429dff2',
                         '54135bf5e4b08f3d2429dfe1','54135bf5e4b08f3d2429dfe3','54135bf5e4b08f3d2429dfe8',
                         '54135bf5e4b08f3d2429dfe9','54135bf5e4b08f3d2429dfe6','54135bf5e4b08f3d2429dfdf',
                         '54135bf5e4b08f3d2429dfe4','54135bf5e4b08f3d2429dfe7','54135bf5e4b08f3d2429dfea',
                         '54135bf5e4b08f3d2429dfeb','54135bf5e4b08f3d2429dfed','54135bf5e4b08f3d2429dfee',
                         '54135bf5e4b08f3d2429dff4','54135bf5e4b08f3d2429dfe0','54135bf5e4b08f3d2429dfdd',
                         '54135bf5e4b08f3d2429dff6','54135bf5e4b08f3d2429dfef','54135bf5e4b08f3d2429dff0',
                         '54135bf5e4b08f3d2429dff1','54135bf5e4b08f3d2429dfde','54135bf5e4b08f3d2429dfec']

In [ ]:
radius = 2000
limit = 200



Let's get the number of Indian restaurants around the Neighbourhood

In [ ]:
csv = False
try:
    venues_df = pd.read_csv('venues_df.csv')
    csv = True
except:
    pass

In [ ]:
if not csv:
    venues = []
    for lat, lng, borough in zip(London_df['Lat'], London_df['Lng'],London_df['Borough']):
        for i in indian_Restaurant:
            url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
                        client_id, client_secret, version, lat,lng ,i, radius, limit)
            result = requests.get(url).json()

            for venue in result['response']['groups'][0]['items']:
                venues.append((borough,lat,lng,
                                      venue['venue']['name'],
                                      venue['venue']['categories'][0]['name'],
                                      venue['venue']['location']['lat'],
                                      venue['venue']['location']['lng'],
                                      venue['venue']['location']['distance']))
    venues_df = pd.DataFrame(venues,columns=['Borough','lat','lng','name','categori','venue_lat','vanue_lng','distance'])
    venues_df.to_csv(r'C:\Users\Shrirang\Desktop\Python Practice\Online Courses\IMB Data Science Cousre\Capstone_Project\Coursera_Capstone\Capstone_project\venues_df.csv', index=False)
    

In [ ]:
venues_df = venues_df.merge(London_df[['Borough','Area','Population']],on = 'Borough')

In [ ]:
venues_df

In [ ]:
#lets get the number of the restaurants per sq.mil of area, 
London_df['Indian_rest_count'] = venues_df.groupby('Borough').count().reset_index()['lat']

#London_df['rest_per_area'] = London_df['Area']/London_df['Indian_rest_count']
#London_df['rest_per_people'] = London_df['Population']/London_df['Indian_rest_count']

In [ ]:
London_df.head(5)

In [ ]:
print('Average Area of borough is                           {:.2f} sq mi'.format(London_df["Area"].mean()))
print('Average Population per borough is                    {:.2f} '.format(London_df["Population"].mean()))
print('Average Indian population per borough is             {:.2f} '.format(London_df["Indian_population"].mean()))
print('Average distance of the borough from city center is  {:.2f} km '.format(London_df["Dist_from_center"].mean()))
print('Average restaurants per borough is                   {:.2f} '.format(London_df["Indian_rest_count"].mean()))

In [ ]:
for i in ['Area','Population','Indian_population','Dist_from_center','Indian_rest_count']:
    fig = plt.figure(figsize=(16,11))
    ax = fig.add_axes([0,0,1,1])
    ax.barh(London_df['Borough'],London_df[i], alpha= 0.9)
    ax.set_ylabel('Borough',fontsize=18)
    ax.set_xlabel(i,fontsize=18)

    #for tick in ax.get_xticklabels():
        #tick.set_rotation(90)
        


In [ ]:
London_df[["Indian_population","Indian_rest_count"]].corr()

Now get the top 100 venues from neighborhood


In [ ]:
top_100 = False
try:
    top_100_df = pd.read_csv("top_100.csv")
    top_100 = True
except:
    pass
    

In [ ]:
if not top_100:
    top_100 = []
    for borough,lat, lng in zip(London_df['Borough'],London_df['Lat'], London_df['Lng']):

        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            client_id,
            client_secret,
            version,
            lat,
            lng,
            2000, 
            100)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        for top in results:
            top_100.append((
                borough,
                lat, 
                lng, 
                top['venue']['name'], 
                top['venue']['location']['lat'], 
                top['venue']['location']['lng'],  
                top['venue']['categories'][0]['name']))
    top_100_df = pd.DataFrame(data = top_100, columns=['Borough','Lat','Lng','name','lat','lng','category'])
    top_100_df.to_csv(r'C:\Users\Shrirang\Desktop\Python Practice\Online Courses\IMB Data Science Cousre\Capstone_Project\Coursera_Capstone\Capstone_project\top_100.csv', index=False)

In [ ]:
top_100_df.head(5)

## 3. Methodology

We will use area population and venues to divide the Boroughs. We will make clusters of boroughs which has same qualities. For that we will use unsupervised machine learning algorithm K-mean clustering. Also, we will use Elbow method and Sillihoette score to get K values. After getting we will narrow down our analysis to select the Borough which has high Indian origin population, low number of Indian restaurants and it should be as possible as close to city centre. With these criteria we will find the location. 

In [ ]:
top_100_df[['Borough','category']].groupby('Borough').count()

There are 282 unique categories present

In [ ]:
print('There are {} unique categories present in Borough'.format(len(top_100_df['category'].unique())))

In [ ]:
top_100_dummy = pd.get_dummies(top_100_df[['category']],prefix="", prefix_sep="")

In [ ]:
top_100_dummy.head(15)

In [ ]:
top_100_dummy.shape


In [ ]:
# add borough column to top_100_dummy
top_100_dummy['Borough'] = top_100_df['Borough']

#rearrange the column
column = top_100_dummy.columns.tolist()

#rearrange the column
column = top_100_dummy.columns.tolist()

top_100_dummy = top_100_dummy[column[-1:] + column[:-1]]

In [ ]:
top_100_dummy.shape

In [ ]:
top_100_dummy.head(5)

In [ ]:
top_100_dummy_grouped = top_100_dummy.groupby('Borough').mean().reset_index()

In [ ]:
top_100_dummy_grouped

In [ ]:
top_100_dummy_grouped.shape

Get the top 10 commom place from each of the borough 

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Borough']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = top_100_dummy_grouped ['Borough']


for ind in np.arange(top_100_dummy_grouped.shape[0]):
    row_categories = top_100_dummy_grouped.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    borough_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]





In [ ]:
borough_venues_sorted

In [ ]:
London_df.head()

Lets add the popuation, area, and distance from center to make a clusters 

In [ ]:
top_100_dummy_grouped['Area'] = London_df['Area']
top_100_dummy_grouped['Population'] = London_df['Population']
top_100_dummy_grouped['Dist_from_center'] = London_df['Dist_from_center']
top_100_dummy_grouped['Indian_population'] = London_df['Indian_population']
top_100_dummy_grouped['Indian_rest_count'] = London_df['Indian_rest_count']

In [ ]:
top_100_dummy_grouped.head()

In [ ]:
# Lets re arrange the columns of data set

column = top_100_dummy_grouped.columns.tolist()
columns_new = column[:1] + column[-5:] + column[1:-5]
top_100_dummy_grouped = top_100_dummy_grouped[columns_new]

In [ ]:
top_100_dummy_grouped.head()

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
top_100_dummy_grouped_scaled = RobustScaler().fit_transform(top_100_dummy_grouped.iloc[:,1:])

In [ ]:
top_100_dummy_grouped_scaled = pd.DataFrame(data = top_100_dummy_grouped_scaled, 
                                            columns=top_100_dummy_grouped.columns.tolist()[1:])

In [ ]:
top_100_dummy_grouped_scaled

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score

In [ ]:
intertias = {}
silhouette_scores= {}
klist = range(2,15)
for k in klist:
    kmeans = KMeans(n_clusters= k ,init='k-means++' , n_init=10).fit(top_100_dummy_grouped_scaled.iloc[:,:5])
    label = kmeans.predict(top_100_dummy_grouped_scaled.iloc[:,:5])
    intertias[k] = kmeans.inertia_
    silhouette_scores[k] = silhouette_score(X = top_100_dummy_grouped_scaled.iloc[:,:5],labels=label)

In [ ]:
silhouette_scores 

In [ ]:
plt.plot(klist, intertias.values(), 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
plt.plot(klist, silhouette_scores.values(), 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Silhouette score for k')
plt.show()

In [ ]:
# As we can see k=4 gives good silhouette score , we will use this for our score

In [ ]:
kmeans = KMeans(n_clusters= 4 ,init='k-means++' , n_init=10).fit(top_100_dummy_grouped_scaled.iloc[:,:5])
label = kmeans.predict(top_100_dummy_grouped_scaled.iloc[:,:5])

In [ ]:
London_df['cluster_labels'] = label

In [ ]:
# map of boroughes with clusters_labes  
map_clusters = folium.Map(location= coordinated_centerlondon, zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i+x+(i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, borough, cluster in zip(London_df['Lat'], London_df['Lng'], London_df['Borough'], London_df['cluster_labels']):
    label = folium.Popup(str(borough) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2.5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
London_df[London_df['cluster_labels'] == 0]

In [ ]:
London_df[London_df['cluster_labels'] == 1]

In [ ]:
London_df[London_df['cluster_labels'] == 2]

In [ ]:
London_df[London_df['cluster_labels'] == 3]

In [ ]:
London_df[London_df['Borough'] == 'Redbridge']

Cluster 3 has more number of the Indian population , compaired with other clusters, but has low number of the Indian restaurants unlike cluster 1,
Redbridge is looking very good for opening new retaurants because , it has very high numbers of Indian population(3 times than average), also it's with in 5 km from city center , More importantly it has only one Indian restaurant.
Lets explore more the Redbridge

In [ ]:
redbridge = top_100_df[top_100_df['Borough'] == 'Redbridge']

In [ ]:
redbridge.shape

In [ ]:
redbridge_rest = redbridge[redbridge['category'].str.lower().str.contains('restaurant')].reset_index()

In [ ]:
redbridge_rest

## 4. Result
Cluster number 3 seems to be more interesting. It has more Indian population and has low number of Indian restaurants Unlike the other cluster, if we narrow down our analysis, we can see that Redbridge has Indian population around 32% of the total population. Redbridge has 27 restaurants in that only one is Indian. Additional to that it is very close to city centre around 4.6 Km. That is why Redbridge is very promising area to open the new Indian restaurant.

In [ ]:
map_la = folium.Map(location = (51.475773,-0.080698), zoom_start = 13)
folium.Marker((51.475773,-0.080698), popup='Redbridge').add_to(map_la)
for lat, lng in zip(redbridge_rest['lat'], redbridge_rest['lng']):
    #folium.CircleMarker([lat, lng], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_la)
    folium.Circle([lat,lng], radius=3,color ="blue").add_to(map_la)
map_la

## 5 Conclusion <a name="conclusion"></a>

London has good number Indian, Pakistani and Bangladeshi people. It is good to open an Indian restaurant in anywhere in London however, the location of the restaurant is key point in business. So, we consider some criteria to open the Indian restaurant.
And according to our analysis, Redbridge borough is very good place to open the new Indian restaurant
